In [12]:

import sys
import os

# Add the 'Backend' folder to sys.path
sys.path.append(os.path.abspath("../Backend/src"))


In [4]:
%pwd

'/Users/tejeshboppana/Desktop/Finance manager/NewprojectDS/Backend/research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'/Users/tejeshboppana/Desktop/Finance manager/NewprojectDS/Backend'

In [26]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file: Path
    unzip_dir: Path

In [16]:
from FinanceManager.constants import *
from FinanceManager.utils.common import read_yaml, create_directories

In [ ]:
!pip install python-box
!pip install ensure

In [28]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir, 
            source_url = config.source_url,  # Corrected spelling
            local_data_file = config.local_data_file, 
            unzip_dir = config.unzip_dir 
        )

        return data_ingestion_config

    



In [20]:
import os
import urllib.request as request
import zipfile
from FinanceManager import logger
from FinanceManager.utils.common import get_size

In [30]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [31]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-27 13:53:22,457: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-27 13:53:22,459: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-27 13:53:22,461: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-27 13:53:22,464: INFO: common: created directory at: artifacts]
[2025-03-27 13:53:22,464: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-27 13:53:23,063: INFO: 1233918453: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 20690
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "860e4e445187d0210c289e14adfa144106d65d5011c3f04cea9ffe46ff1ae04e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1350:3590AF:CB86E:109630:67E59092
Accept-Ranges: bytes
Date: Thu, 27 